# Orbits of planets and spacecraft
***

*EUHFORIA* requires data of the orbits of spacecraft/planets in order to be able to extract data at the spacecraft position from the simulation. This notebook shows the essential steps of how the spacecraft position information is stored and accessed by *EUHFORIA*.

## 1. Getting orbit data

The first step is to get data of the orbit of the spacecraft/planet etc. that one is interested in. In particular, what is needed is the position of the object in HEEQ coordinates as a function of time. For most objects, so called SPICE kernels are provided by NASA/ESA etc. We will use these together in the context of the [SpiceyPy](https://github.com/AndrewAnnex/SpiceyPy) package to compute the orbit data. Another usefull resource is the
[NASA HelioWeb](https://omniweb.gsfc.nasa.gov/coho/helios/heli.html) tool.

### Downloading basic SPICE kernels

Planet ephemeris kernel file:

In [ ]:
!wget https://naif.jpl.nasa.gov/pub/naif/generic_kernels/spk/planets/de430.bsp -P spice_kernels/

Orientation data:

In [ ]:
!wget https://naif.jpl.nasa.gov/pub/naif/generic_kernels/pck/pck00010.tpc -P spice_kernels/

Leap second data:

In [ ]:
!wget https://naif.jpl.nasa.gov/pub/naif/generic_kernels/lsk/naif0012.tls -P spice_kernels/

Heliospheric and geomagnetic coordinate system definitions:

In [ ]:
!wget http://proba2.sidc.be/aux/data/spice/kernels/fk/heliospheric.tf -P spice_kernels/

In [ ]:
!wget http://proba2.sidc.be/aux/data/spice/kernels/fk/geomagnetic.tf -P spice_kernels/

Earth data:

In [ ]:
!wget https://naif.jpl.nasa.gov/pub/naif/generic_kernels/pck/earth_720101_070426.bpc -P spice_kernels/

In [ ]:
!wget https://naif.jpl.nasa.gov/pub/naif/generic_kernels/pck/earth_070425_370426_predict.bpc -P spice_kernels/

### Creating the SPICE meta file

After the kernels have been downloaded, a meta-file for loading them needs to be created:

In [ ]:
%%writefile spice_kernels/meta.txt

\begindata
KERNELS_TO_LOAD=(
 'spice_kernels/de430.bsp',
 'spice_kernels/pck00010.tpc',
 'spice_kernels/naif0012.tls',
 'spice_kernels/heliospheric.tf',
 'spice_kernels/geomagnetic.tf',
 'spice_kernels/earth_070425_370426_predict.bpc',
 'spice_kernels/earth_720101_070426.bpc')
\begintext

## 2. Computing the orbits

After the needed kernels have been downloaded, we can start using them to compute the orbits using SpiceyPy. 

In [ ]:
import spiceypy as spice
import datetime
import numpy as np

In [ ]:
astronomical_unit  = 149597870700.0

Load the kernels:

In [ ]:
spice.furnsh("spice_kernels/meta.txt")

Define function to compute HEEQ coordinates of an object:

In [ ]:
def get_heeq(body, dates):
    """Computes the HEEQ coordinates for the given body at the given times.
    
    Args:
        body (str) : Name of the body
        dates      : List of datetime objects for which to computes the position
    Returns:
        HEEQ tuple of r [m], lat [rad], lon [rad] 
    """
    
    num_dates = len(dates)
        
    r     = np.zeros(num_dates)
    lon   = np.zeros(num_dates)
    lat   = np.zeros(num_dates)
    
    for idx, date in enumerate(dates):
    
        # Compute epoch time
        et = spice.str2et(datetime.datetime.strftime(date, format="%Y-%m-%dT%H:%M:%S"))

        # Compute position
        pos_and_vel, light_travel_time = spice.spkezr(body, et, 'HEEQ', 'None', 'Sun')
        
        # Position in spherical coordinates
        heeq_r_km, lon[idx], lat[idx] = spice.reclat(pos_and_vel[0:3])

        # Convert unit to m
        r[idx] = heeq_r_km*1e3
                
    return r, lat, lon

Define the dates of interest:

In [ ]:
dates = [datetime.datetime(1975, 1, 1, 12, 0) + datetime.timedelta(days=x) for x in range(0, 21915)]

Let's compute and save the position for a selection of objects for the given dates. Note that I here use the barycenter position simply to avoid downloading further kernels.

In [ ]:
for obj in ('Mercury', 'Venus', 'Earth', 'Mars', 'Jupiter', 'Saturn', 'Uranus', 'Neptune', 'Pluto'):
    
    # Get HEEQ position
    r, lat, lon = get_heeq(obj + ' barycenter', dates)
    
    # Save to file
    np.savez("orbit_data/"+obj.lower(), dates=dates, r=r, lat=lat, lon=lon)

## STEREO

STEREO SPICE kernels are located at https://hesperia.gsfc.nasa.gov/ssw/stereo/gen/data/spice

Download definite (i.e. past ?) data: 

In [ ]:
!wget -r --no-parent -nH -nd -A '*.bsp' https://hesperia.gsfc.nasa.gov/ssw/stereo/gen/data/spice/depm/ahead/ -P spice_kernels/

In [ ]:
!wget -r --no-parent -nH -nd -A '*.bsp' https://hesperia.gsfc.nasa.gov/ssw/stereo/gen/data/spice/depm/behind/ -P spice_kernels/

Download predictive data (*NOTE*: The name of the file will change with time)

In [ ]:
!wget https://hesperia.gsfc.nasa.gov/ssw/stereo/gen/data/spice/epm/ahead/ahead_2017_061_5295day_predict.epm.bsp -P spice_kernels/

In [ ]:
!wget https://hesperia.gsfc.nasa.gov/ssw/stereo/gen/data/spice/epm/behind/behind_2009_049_definitive_predict.epm.bsp -P spice_kernels/

### Load and save data for STA

In [ ]:
import glob

In [ ]:
for files in glob.glob("spice_kernels/ahead*.bsp"):
    spice.furnsh(files)

In [ ]:
dates = [datetime.datetime(2006, 10, 10, 12, 0) + datetime.timedelta(days=x) for x in range(0, 8766)]

In [ ]:
r, lat, lon = get_heeq('STEREO ahead', dates)

In [ ]:
np.savez("orbit_data/sta", dates=dates, r=r, lat=lat, lon=lon)

### Load and save data for STB

In [ ]:
for files in glob.glob("spice_kernels/behind*.bsp"):
    print files
    spice.furnsh(files)

In [ ]:
dates = [datetime.datetime(2006, 10, 10, 12, 0) + datetime.timedelta(days=x) for x in range(0, 3626)]

In [ ]:
r, lat, lon = get_heeq('STEREO behind', dates)

In [ ]:
np.savez("orbit_data/stb", dates=dates, r=r, lat=lat, lon=lon)